# Useful Links
Useful links:
* Export job to GCS: https://blog.getcensus.com/3-ways-to-export-csv-files-from-google-bigquery/
* Export job to GCS: https://hevodata.com/learn/export-bigquery-table-to-csv/#met3
* Export job to GCS: https://cloud.google.com/bigquery/docs/samples/bigquery-extract-table#bigquery_extract_table-python
* Query BQ via Python: https://sophiamyang.github.io/DS/dataaccess/bigquery/bigquery.html
* Query via Python: https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries?hl=de#run_the_query

* Query GBQ with pandas.gbq: https://stackoverflow.com/questions/44647310/convert-bigquery-results-to-pandas-data-frame
* Doc pandas-gbq: https://pandas-gbq.readthedocs.io/en/latest/install.html#conda
* Google doc zu pandas-gbq: https://cloud.google.com/bigquery/docs/pandas-gbq-migration?hl=de#conda
* Running Parametrized queries: https://cloud.google.com/bigquery/docs/parameterized-queries
* .to_dataframe doc: https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe
* Get BQ table as dataframe: https://cloud.google.com/bigquery/docs/samples/bigquery-list-rows-dataframe
* globals().clear()


# Packages

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery # when buggy: 'conda remove protobuf'
import os
import pyarrow


In [2]:
print(bigquery.__version__)

3.0.1


# Set env variable for credentials

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/marcvendramet/Desktop/HSG/Master/MA/08_Data/03_BQKey/mevtest-32e3980294f1.json"

In [4]:
client = bigquery.Client()

# Build Query Masterset

## Define query function

In [5]:
my_query = '''
SELECT *


FROM `mevtest.MEVextract.UniswapV2_transactions_download_v4`
WHERE block_number BETWEEN  @lower_bound AND @upper_bound 

ORDER BY block_number ASC, transaction_index ASC, log_index ASC

'''

## Collect results

In [ ]:
bqCols = ['transaction_hash','block_number','transaction_index','log_index','from_address','to_address',
                            'trans_fromAddress','trans_toAddress', 'poolId', 'tf_tokenAddress', 'tokenName',
                            'value','trans_gasPrice','gasUsed','effective_gas_price']

In [ ]:
dfMain = pd.DataFrame(columns = bqCols)

# Collect loop

## Collect manual

In [6]:
counter = 1
lowerB = 10093070
upperB = lowerB + 100000

In [76]:

job_config1 = bigquery.QueryJobConfig(
query_parameters=[
        bigquery.ScalarQueryParameter("lower_bound", "INT64", lowerB),
        bigquery.ScalarQueryParameter("upper_bound", "INT64", upperB),
    ]
    )
    
query_job = client.query(my_query, job_config=job_config1)
    
dfTemp = query_job.to_dataframe()
# create path variable
# save to.csv or parquet on ssd OK
dfTemp.to_parquet(f"/Volumes/Extreme SSD/02_UniswapV2Transactions/UniswapV2Transactions_{counter}.par")
# collect params from dfTemp andd update counter for new query OK
counter = counter + 1
lastEntry = dfTemp["block_number"].max().item() # OR: dfMain["block_number"].iloc[-1]
lowerB = lastEntry+1
upperB = lastEntry+100001

KeyboardInterrupt: 

In [75]:
len(dfTemp.iloc[:,0]) >=1
print(lowerB, upperB, counter)

10393072 10493072 4


In [37]:
dfTemp.to_csv(f"/Volumes/Extreme SSD/02_UniswapV2Transactions/UniswapV2Transactions_2.csv")

In [30]:
dfTemp1 = dfTemp

In [73]:
lastEntry= lastEntry.item()
lowerB=lowerB.item()
upperB=upperB.item()

In [69]:
type(lastEntry.item())

int

## Automatic loop (full)

In [7]:
while True:
    # len(dfTemp.iloc[:,0]) >=1
    print(lowerB, upperB)
    job_config1 = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("lower_bound", "INT64", lowerB),
        bigquery.ScalarQueryParameter("upper_bound", "INT64", upperB),
    ]
    )
    
    query_job = client.query(my_query, job_config=job_config1)
    
    dfTemp = query_job.to_dataframe()
    
    if dfTemp["block_number"].max() >= 12500188:
        dfTemp.to_parquet(f"/Volumes/Extreme SSD/02_UniswapV2Transactions/UniswapV2Transactions_{counter}.par")
        break
    else:
        # create path variable
        # save to.csv or parquet on ssd OK
        dfTemp.to_parquet(f"/Volumes/Extreme SSD/02_UniswapV2Transactions/UniswapV2Transactions_{counter}.par")
        # collect params from dfTemp andd update counter for new query OK
        counter = counter + 1
        lastEntry =dfTemp["block_number"].max().item() # OR: dfMain["block_number"].iloc[-1]
        lowerB = lastEntry+1
        upperB = lastEntry+100001
        print(lastEntry)

10093070 10193070
10193070
10193071 10293071
10293070
10293071 10393071
10393071
10393072 10493072
10493072
10493073 10593073
10593073
10593074 10693074
10693074
10693075 10793075
10793075
10793076 10893076
10893076
10893077 10993077
10993077
10993078 11093078
11093078
11093079 11193079
11193079
11193080 11293080
11293080
11293081 11393081
11393081
11393082 11493082
11493082
11493083 11593083
11593083
11593084 11693084
11693084
11693085 11793085
11793085
11793086 11893086
11893086
11893087 11993087
11993087
11993088 12093088
12093088
12093089 12193089
12193089
12193090 12293090
12293090
12293091 12393091
12393091
12393092 12493092
12493092
12493093 12593093


In [ ]:
# Add nice print outs of status, loop number, part of loop its in, etc

In [10]:
dfTemp['block_number'].max()

12500188

In [11]:
dfTemp.shape

(229801, 15)

# Query: GasPrice Threeshold data set

In [5]:
my_query_GP = '''
SELECT *  
FROM `mevtest.MEVextract.GasPrice_threshold4`

WHERE block_number BETWEEN  11296630 AND 12500188 

'''

In [8]:
11296630-10093070

1203560

In [6]:

query_job = client.query(my_query_GP)
    





In [17]:
dfTemp = query_job.to_dataframe()


In [18]:
dfTemp['block_number'].max()

12500188

In [19]:
dfTemp.to_parquet(f"/Volumes/Extreme SSD/01_GasData/GasPriceThreshold_02.par")

In [14]:
a = (12500188-10093070)/2
10093070 + a

11296629.0

# Archive
## Query function
def query_mev():
    #client = bigquery.Client()
    query_job = client.query(
    """
    SELECT * 
    FROM `mevtest.MEVextract.LPs_UniswapV2`
    """)
    #results = query_job.result()
    #yield from results

## Potential eeasy way to store as pandas df
dfTest = query_job.to_dataframe() https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe

## unpack function def (used before .to_dataframe())
def query_unpack():
    results = query_job.result()
    yield from results

## Collect results and add to df
results2 = query_unpack()
dftemp = []
for row in results2:
    dftemp.append([
        row.block_number
    ])

dftemp = pd.DataFrame(dftemp, columns = ['block_number'])
dfMain = dfMain.append(dftemp)

## Query loop old version
while True:
    job_config1 = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("lower_bound", "INT64", lowerB),
        bigquery.ScalarQueryParameter("upper_bound", "INT64", upperB),
    ]
    )
    
    query_job = client.query(my_query, job_config=job_config1)
    
    dfTemp = query_job.to_dataframe()
    
    if len(dfTemp.iloc[:,0]) <=1:
        break
    else:
        # create path variable
        # save to.csv or parquet on ssd OK
        dfTemp.to_parquet(f"/Volumes/Extreme SSD/02_UniswapV2Transactions/UniswapV2Transactions_{counter}.par")
        # collect params from dfTemp andd update counter for new query OK
        counter = counter + 1
        lastEntry =dfTemp["block_number"].max() # OR: dfMain["block_number"].iloc[-1]
        lowerB = lastEntry+1
        upperB = lastEntry+100001
        # check for max block
        if dfTemp["block_number"].max() >= 12500188:
            break
        else:
            # clean dfTemp
            dfTemp1 =dfTemp
            dfTemp =pd.DataFrame()
        
        
        
        
    